In [ ]:
import fitz
import unicodedata
import unicodedata
from glob import glob
import pickle
import os
import re

paths = glob('../../data/*/*교시.pdf')
#display(paths)
path = paths[-1]

In [ ]:
def pdf2text(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()

    lines = text.split('\n')
    i = 0
    questions = []
    start = None
    end = None
    for i, line in enumerate(lines):
        if len(line) > 4 and unicodedata.category(line[0]) == 'Nd' and line.split()[0][-1] == '.':
            start = i
        if len(line) > 0 and line[0] == '⑤':
            end = i
            while end+1 < len(lines) and len(lines[end+1]) > 0:
                if lines[end+1][0] == '[' or unicodedata.category(lines[end+1][0]) == 'Nd' or lines[end+1][1:3] == '교시':
                    break
                else: end += 1
            sentence = lines[start:end+1]
            questions.append(sentence)
    return questions

In [ ]:
def text2array(questions):
    new_list = []
    for q, item in enumerate(questions):
        choice = []
        etc = []
        ques = []
        is_choice = False
        ques_end = False
        is_cont = False
        for i, line in enumerate(item):
            if line[-1] in '?>' or line[-2:]=='? ' or '? (' in line:
                ques_index = i
                if '? (' in line and ')' in item[i+1]:
                    ques_index += 1
                ques_end = True
                continue
            elif ques_end and ques_index < i:
                if line[0] in '①②③④⑤':
                    is_choice = True
                    choice.append(line)
                elif '국가시험' in line:
                    continue
                elif is_choice:
                    if line[-1] in '- ':
                        choice[-1] += line
                    else:
                        choice[-1] += line
                else:
                    if line[0] == '•' or line[0] == '[':
                        is_cont = True
                        etc.append(line)
                    elif line[0] != '•' and line[0] != '[' and is_cont:
                        etc[-1] += ' ' + line
                        test = False
                    elif len(line) < 10 and len(etc) > 0:
                        etc[-1] += ', ' + line
                    else:
                        etc.append(line)

        ques = ''.join(item[:ques_index+1])
        new_list.append([ques, etc, choice])
    return new_list

In [ ]:
def couple_text(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()

    lines = text.split('\n')
    i = 0
    questions = []
    start = None
    end = None
    pattern = r"\[\d+～\d+\]"
    start_tag = ''
    for i, line in enumerate(lines):
        matches = re.search(pattern, line)
        if matches:
            #print(line)
            start_num = matches.group().split('～')[0][1:]
            #print(start_num)
            start_tag = f'{start_num}.'
            start = i
        elif line[0:len(start_tag)] == start_tag and start_tag != '':
            #print(line)
            end = i
            sentence = lines[start:end]
            #display(sentence)
            questions.append(sentence)
    return questions

In [ ]:
def couple_array(questions):
    new_list = []
    for q, item in enumerate(questions):
        text = ''
        for i, line in enumerate(item):
            if i == 0:
                text += (line+'\n')
                temp = line.split(']')[0].split('～')
                start_num = int(temp[0][1:])
                end_num = int(temp[-1])
                continue
            else:
                if line[0] in '•[':
                    text += ('\n' + line + '\n')
                    continue
                elif line[-1] in '- ':
                    text += (line + '')
                    continue
                else:
                    text += (line + '\n')
                    continue
        new_list.append([start_num, end_num, text])
    return new_list

In [ ]:
def run(paths):
    if not os.path.exists('save'):
        os.makedirs('save')
    for path in paths:        
        questions = pdf2text(path)
        array = text2array(questions)

        couple = couple_text(path)
        couple_array_ = couple_array(couple)
        for coup_ques in couple_array_:
            temp1 = (coup_ques[2] + '\n' + array[coup_ques[0]-1][0])
            temp2 = (coup_ques[2] + '\n' + array[coup_ques[1]-1][0])
            array[coup_ques[0]-1][0] = temp1
            array[coup_ques[1]-1][0] = temp2
        
        print(len(array))

        filename = os.path.basename(path).split()
        picklename = f'{filename[0][0:4]}_{filename[1][1:3]}_{filename[-1][0]}.pkl'
        with open(f'save/{picklename}', 'wb') as f:
            pickle.dump(array, f)

run(paths)

In [ ]:
saves = glob('save/*.pkl')

with open(saves[3], 'rb') as f:
    loaded_data = pickle.load(f)
f.close()

len(loaded_data)

In [ ]:
loaded_data

In [ ]:
print(loaded_data[50][0])

In [ ]:
print(loaded_data[62][0])